In [2]:
import numpy as np
import torch 

from python_tsp.heuristics import solve_tsp_local_search, solve_tsp_simulated_annealing



class Guided_local_Search():
    def make_edge_index_and_regret(output,edge_index):
        output = torch.squeeze(output, 1)
        output1= output.detach().numpy()
        node_index2 = []
        node_index1 = []
        node_index3 = []
        regret_values = []
       

        #nodes=[]
        #for node in L.nodes(data=True):
            #nodes.append(node[0])
        for i in range(len(edge_index)):
            node_index3.append(edge_index[i])
            regret_values.append(output1[i])


        for i in range(len(edge_index)):
            node_index2.append(edge_index[i][1])
            node_index1.append(edge_index[i][0])
            node_index3.append((node_index2[i], node_index1[i]))
            regret_values.append(output1[i])

        #print(node_index3)
        return node_index3,regret_values

    def find_the_value(edges,nodes):
        return nodes.index((edges[0],edges[1]))

    #node_index3,regret_values = make_edge_index_and_regret(output,edge_index)
    #print(node_index3)



    #edw einai mia apli me8odos get() gia na mporoume na xwrisoume apo (1,2) dedomeno se (1) kai (2)
    def get_nodes(edge):
        return edge[0], edge[1]

    def edge_on_tour(edge, tour):
        tour_edges = Guided_local_Search.tour_to_table(tour)
        return edge in tour_edges or (edge[1], edge[0]) in tour_edges

    #edw kanei ar8isi oles tis apostaseis pou epilegonte san diadromh gia na bgalei thn sunoliki apostash
    def calculate_distance(path, graph):
        distance = 0
        for i in range(len(path) - 1):
            distance += graph[path[i]][path[i + 1]]
        distance += graph[path[-1]][path[0]]  # Return to starting point
        return distance

    #Edw exoume thn prwti me8odo pou einai Greedy Nearest Neighbor sigkekrimena epilegi me8odika tous kombous me thn seira pou
    #den exoume episkeftei kai episis exoun to mikrotero regret
    def greedy_nearest_neighbor(regret, edges, distances):
        num_cities = distances.shape[0]
        visited = [False] * num_cities
        tour = [0]  # Start from city 0
        current_city = 0
        visited[current_city] = True
        sum_tour_d = 0

        for _ in range(num_cities - 1):
            nearest_city = None
            min_distance = float('inf')

            for next_city in range(len(regret)):
                poli1, poli2 = Guided_local_Search.get_nodes(edges[next_city])
                if visited[poli1] == False and visited[poli2] == True and current_city == poli2:
                    if regret[next_city] < min_distance:
                        nearest_city = poli1
                        min_distance = regret[next_city]

                elif visited[poli2] == False and visited[poli1] == True and current_city == poli1:
                    if regret[next_city] < min_distance:
                        nearest_city = poli2
                        min_distance = regret[next_city]

            #sum_tour.append(min_distance)
            tour.append(nearest_city)
            visited[nearest_city] = True
            current_city = nearest_city
            sum_tour_d = Guided_local_Search.calculate_distance(tour, distances)

        return tour, sum_tour_d  #sum_tour, 

    #Se afti thn me8odo ksana enonoume ta dedomena, apo to na exoume enan pinaka [1,2,3,4] exoume [(1,2), (2,3), (3,4)]
    def tour_to_table(tour):
        table = []
        for i in range(len(tour) - 1):
            pair = (tour[i], tour[i + 1])
            table.append(pair)
        return table

    def penalize_edges(regret_values, penalties, edges,  node_index3):
        utilities = np.zeros(len(edges))

        for i in range(len(edges)):
            edge = edges[i]
            edge_idx = Guided_local_Search.find_the_value(edge, node_index3)
            utilities[i] = regret_values[edge_idx] / (1 + penalties[edge_idx])

        max_utility = np.max(utilities)
        max_utility_indices = np.where(utilities == max_utility)[0]
        max_utility_edges = [edges[i] for i in max_utility_indices]

        for edge in max_utility_edges:
            edge_idx = Guided_local_Search.find_the_value(edge, node_index3)
            penalties[edge_idx] += 1
        #print(node_index3)
        return penalties, max_utility_edges
    #Auth h me8odos pernei dio ari8mous apo ton pinaka (dio kombous apo thn diadromi) kai tous allazei 8esi sth diadromi
    def two_opt_swap(path, i, j):
        new_path = path[:i] + path[i:j+1][::-1] + path[j+1:]
        return new_path

    #Edw exoume thn me8odo 2opt swap dld pernei dio kombous apo thn diadromi kai tous allazei metaksei tous 
    #o algorithoms stamataei kai molis bre8ei kaluteri diadromo (mikroteri apostasi)
    def two_opt_local_search(path, graph, best_distance):
        n = len(path)
        best_path = path
        improved = True

        while improved:
            improved = False
            for i in range(1, n - 2):
                for j in range(i + 1, n):
                    if j - i == 1:  # No point in reversing if i and j are adjacent
                        continue
                    new_path = Guided_local_Search.two_opt_swap(best_path, i, j)
                    new_distance = Guided_local_Search.calculate_distance(new_path, graph)

                    if new_distance < best_distance:
                        best_distance = new_distance
                        best_path = new_path
                        improved = True
                        break
                if improved:
                    break

        return best_path, best_distance

    #Afti i me8odos allazei 8esi mono se enan kombo sth diadromi gia na beltiosi thn diadromi 
    def relocate_algorithm(distance_matrix, old_route, best_distance, max_iterations=1000):
        num_cities = len(old_route)
        current_route = old_route[:]  # Copy the initial route
        current_distance = best_distance

        for _ in range(max_iterations):
            city1, city2 = np.random.choice(num_cities, size=2, replace=False)

            # Ensure city1 is always before city2 in the tour
            if city1 > city2:
                city1, city2 = city2, city1

            new_route = current_route[:city1] + current_route[city1+1:]
            new_route = new_route[:city2] + [current_route[city1]] + new_route[city2:]
            new_distance = Guided_local_Search.calculate_distance(new_route, distance_matrix)

            if new_distance < current_distance:
                current_route = new_route
                current_distance = new_distance

        return current_route, current_distance


    def local_search_on_penalized_edges(solution, best_distance, max_utility_edges, node_index3, regret_values, distances):
        improved_tour = solution[:]
        improved_dis = best_distance

        #print("Initial Solution:", improved_tour)
        #print("Penalized Edges:", max_utility_edges)

        # Perform 2-opt swaps focusing on penalized edges
        for edge in max_utility_edges:
            #print("Trying to remove edge:", edge, max_utility_edges)
            for i in range(1, len(improved_tour) - 1):
                for j in range(i + 1, len(improved_tour)):
                    if (improved_tour[i], improved_tour[j]) == edge or (improved_tour[j], improved_tour[i]) == edge:
                        new_path = Guided_local_Search.two_opt_swap(improved_tour, i, j)
                        new_distance = Guided_local_Search.calculate_distance(new_path, distances)
                        #print("Old Distance:", best_distance, "New Distance:", new_distance)
                        if new_distance < improved_dis:
                            improved_tour = new_path
                            improved_dis = new_distance
                            print("Removed edge with 2-opt:", edge, "New Distance:", new_distance)

        # Perform relocation focusing on penalized edges
        for edge in max_utility_edges:
            #print("Trying to relocate node in edge:", edge)
            for i in range(1, len(improved_tour) - 1):
                if improved_tour[i] in edge:
                    for j in range(len(improved_tour)):
                        if j != i and (improved_tour[j], improved_tour[i]) not in max_utility_edges and (improved_tour[i], improved_tour[j]) not in max_utility_edges:
                            new_path, new_distance = Guided_local_Search.relocate_algorithm(distances, improved_tour, improved_dis, improved_tour[i])
                            #print("Old Distance:", best_distance, "New Distance:", new_distance)
                            if new_distance < improved_dis:
                                improved_tour = new_path
                                improved_dis = new_distance
                                print("Relocated node:", improved_tour[i], "New Distance:", new_distance)

        return improved_tour, improved_dis


    def gls(distances, regret_values, node_index3, max_iterations=20):

        initial_solution, initial_distance = Guided_local_Search.greedy_nearest_neighbor(regret_values, node_index3, distances)
        best_solution_t, best_distance_t = Guided_local_Search.two_opt_local_search(initial_solution, distances, initial_distance)
        best_solution, best_distance = Guided_local_Search.relocate_algorithm(distances, best_solution_t, best_distance_t)

        #print("greedy_nearest_neighbor:",initial_solution, initial_distance)
        #print("two_opt_local_search:",best_solution_t, best_distance_t)
        print("relocate_algorithm:",best_solution, best_distance)

        penalties = [0] * len(regret_values)
        for _ in range(max_iterations):
            edges = Guided_local_Search.tour_to_table(best_solution)
            penalties, max_utility_edges = Guided_local_Search.penalize_edges(regret_values, penalties, edges ,  node_index3)
            
            best_solution, best_distance = Guided_local_Search.local_search_on_penalized_edges(best_solution, best_distance, max_utility_edges, node_index3, regret_values, distances)
        #print(penalties, max_utility_edges)
        #print(node_index3)
        return best_solution, best_distance
    
    def tsp_solver(distance_mtx):
        permutation, distance = solve_tsp_local_search(distance_mtx)
        return distance
